In [1]:
import tqdm
import urllib
import os.path
import sqlite3
import pandas as pd
import re
import numpy as np
import requests
import requests_random_user_agent
import threading
from w3lib.url import safe_url_string
from urllib.parse import urlparse, urlunparse
from bs4 import BeautifulSoup as bf


AMAZON_URL = "https://www.amazon.com/s?k={}&i=movies-tv"
AMAZON_URL_HEAD = "https://www.amazon.com"
IMDB_URL =  "https://www.imdb.com/title/tt{}/?ref_=fn_al_tt_1"

MOVIELENS_PATH = './'
OUTPUT_CSV = "./prices.csv"
DATABASE_PATH = "./database.db"
REQUEST_TIMEOUT = 5.0

### Load movies.csv and links.csv

In [2]:
header = ['movieId', 'title', 'genres']
movies_df = pd.read_csv(MOVIELENS_PATH + 'movies.dat', sep='::', dtype=str, engine='python', names=header)
print(movies_df.head())

  movieId                               title                        genres
0       1                    Toy Story (1995)   Animation|Children's|Comedy
1       2                      Jumanji (1995)  Adventure|Children's|Fantasy
2       3             Grumpier Old Men (1995)                Comedy|Romance
3       4            Waiting to Exhale (1995)                  Comedy|Drama
4       5  Father of the Bride Part II (1995)                        Comedy


In [17]:
movies_df.shape[0]

3883

### Create Database and Table

Table structure: movieId (int)，imdbId (varchar30), title (varchar50), pageUrl (varchar300), price (double)

In [3]:
database = sqlite3.connect(DATABASE_PATH, check_same_thread=False)
cursor = database.cursor()
cursor.execute('create table if not exists prices (movieId integer primary key, title varchar(50) NOT NULL, pageUrl varchar(300), price double)')
database.commit()

### Query exist tables

In [10]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Current table: ", cursor.fetchall())

Current table:  [('prices',)]


### Query table structure

In [11]:
cursor.execute("PRAGMA table_info(prices)")
print("Table prices: ", cursor.fetchall())

Table prices:  [(0, 'movieId', 'integer', 0, None, 1), (1, 'title', 'varchar(50)', 1, None, 0), (2, 'pageUrl', 'varchar(300)', 0, None, 0), (3, 'price', 'double', 0, None, 0)]


### Insert movie data if not exist

In [15]:
row_num = movies_df.shape[0]
pbar = tqdm.trange(row_num, position=0, leave=True)
for i in pbar:
    pbar.set_description("Processing row %d" % i)
    row_item = movies_df.iloc[i]
    cursor.execute("SELECT * FROM prices WHERE movieId = ?;", (int(row_item['movieId']),))
    if len(cursor.fetchall()) == 0:
        cursor.execute("INSERT INTO prices (movieId, title, pageUrl, price) VALUES (?, ?, NULL, NULL)",
                       (int(row_item['movieId']), row_item['title']))
        database.commit()

Processing row 3882: 100%|██████████| 3883/3883 [00:06<00:00, 598.67it/s]


### Display first 5 data from database

In [18]:
cursor.execute("SELECT * FROM prices LIMIT 0, 10;")
print(cursor.fetchall())

[(1, 'Toy Story (1995)', None, None), (2, 'Jumanji (1995)', None, None), (3, 'Grumpier Old Men (1995)', None, None), (4, 'Waiting to Exhale (1995)', None, None), (5, 'Father of the Bride Part II (1995)', None, None), (6, 'Heat (1995)', None, None), (7, 'Sabrina (1995)', None, None), (8, 'Tom and Huck (1995)', None, None), (9, 'Sudden Death (1995)', None, None), (10, 'GoldenEye (1995)', None, None)]


In [4]:
def getPageUrl(title):
    url = AMAZON_URL.format(title.replace(" ", "+"))
    url = safe_url_string(url, encoding="utf-8")
    req = urllib.request.Request(url)
    req.add_header('User-Agent', getRandomUserAgent())
    html = urllib.request.urlopen(req, timeout=REQUEST_TIMEOUT)
    obj = bf(html.read(),'html.parser')
    firstItem = obj.body.find_all('div',class_="a-section a-spacing-medium")[0]
    aElement = firstItem.find_all('a', class_="a-link-normal a-text-normal")[0]
    pageUrl = AMAZON_URL_HEAD + aElement['href']
    return pageUrl

def getPageUrlFromIMDB(imdbId):
    url = AMAZON_URL.format(title.replace(" ", "+"))
    req = urllib.request.Request(url)
    req.add_header('User-Agent', getRandomUserAgent())
    html = urllib.request.urlopen(req, timeout=REQUEST_TIMEOUT)
    obj = bf(html.read(),'html.parser')
    firstItem = obj.body.find_all('div',class_="a-section a-spacing-medium")[0]
    aElement = firstItem.find_all('a', class_="a-link-normal a-text-normal")[0]
    pageUrl = AMAZON_URL_HEAD + aElement['href']
    return pageUrl

def getMoviePrice(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', getRandomUserAgent())
    html = urllib.request.urlopen(req, timeout=REQUEST_TIMEOUT)
    obj = bf(html.read(),'html.parser')
    lst = [str(x) for x in obj.body.find_all('button') if 'Buy' in str(x)]
    if len(lst) == 0:
        d = obj.body.find('tr', id='priceblock_ourprice_row').find('span', id='priceblock_ourprice').contents
        return float(str(d[0])[1:])
    else:
        return float(re.findall(r'(?<=\$)\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})', lst[0])[0])

def removeBrackets(title):
    return title[:-7] + " " + title[-5:-1]

def urlEncodeNonAscii(b):
    return re.sub('[\x80-\xFF]', lambda c: '%%%02x' % ord(c.group(0)), b)

def iriToUri(iri):
    parts= urlparse(iri)
    return urlunparse(
        part.encode('idna') if parti==1 else urlEncodeNonAscii(part.encode('utf-8'))
        for parti, part in enumerate(parts)
    )

def runGetPage(movie_list):
    pbar = tqdm.tqdm(range(len(movie_list)), position=0, leave=True)
    for i in pbar:
        try:
            pbar.set_description("Processing row %d" % i)
            m = movie_list[i]
            movie_id = m[0]
            movie_title = m[1]
            page_url = m[2]
            if page_url is None:
                page_url = getPageUrl(removeBrackets(movie_title))
                updatePageUrl(page_url, movie_id)
        except KeyboardInterrupt:
            sys.exit()
            pass
        except Exception as e:
            print("Exception at row %d" % i)
            print(e)
            continue

def runGetPrice(movie_list):
    pbar = tqdm.tqdm(range(len(movie_list)), position=0, leave=True)
    for i in pbar:
        try:
            pbar.set_description("Processing row %d" % i)
            m = movie_list[i]
            movie_id = m[0]
            page_url = m[2]
            movie_price = m[3]
            if page_url is not None and movie_price is None:
                price = getMoviePrice(page_url)
                updatePrice(price, movie_id)
        except KeyboardInterrupt:
            sys.exit()
            pass
        except Exception as e:
            print("Exception at row %d" % i)
            print(e)
            continue
            
def getRandomUserAgent():
    s = requests.Session()
    return s.headers['User-Agent']

def list_split(items, n):
    return [items[i:i+n] for i in range(0, len(items), n)]

lock = threading.Lock()

def updatePageUrl(page_url, movie_id):
    try:
        lock.acquire(True)
        cursor.execute("UPDATE prices SET pageUrl = ? WHERE movieId = ?;", (page_url, movie_id))
        database.commit()
    finally:
        lock.release()
        
def updatePrice(price, movie_id):
    try:
        lock.acquire(True)
        cursor.execute("UPDATE prices SET price = ? WHERE movieId = ?;", (price, movie_id))
        database.commit()
    finally:
        lock.release()

In [99]:
cursor.execute("UPDATE prices SET pageUrl = NULL WHERE imdbId = ?;", ("0112682", ))
database.commit()

In [20]:
getPageUrl(removeBrackets("Grumpier Old Men (1995)"))

'https://www.amazon.com/Grumpier-Old-Men-Jack-Lemmon/dp/B00AUOUG8W/ref=sr_1_1?dchild=1&keywords=Grumpier+Old+Men+1995&qid=1619358956&s=movies-tv&sr=1-1'

In [122]:
imdbId = "0114709"
url = IMDB_URL.format(imdbId)
req = urllib.request.Request(url)
req.add_header('User-Agent', getRandomUserAgent())
html = urllib.request.urlopen(req, timeout=REQUEST_TIMEOUT)
obj = bf(html.read(),'html.parser')
print(obj.body.find_all('button',class_="buybox__button"))

[]


### Load data from database to Dataframe

In [ ]:
output_df = pd.DataFrame(columns=['movieId', 'title', 'pageUrl', 'price'])
cursor.execute("SELECT * FROM prices;")
pbar = tqdm.tqdm(cursor.fetchall(), position=0, leave=True)
for row in pbar:
    pbar.set_description("Processing %d" % 1)
    row_dict = {'movieId': row[0], 'title': row[1], 'pageUrl': row[2], 'price': row[3]}
    output_df = output_df.append(row_dict, ignore_index=True)

Processing 1:  77%|███████▋  | 20920/27278 [01:58<00:37, 170.03it/s]

### Write into CSV file

In [121]:
output_df.to_csv(OUTPUT_CSV, index=False)

In [116]:
getPageUrl('Jumanji 1995')

'https://www.amazon.com/Jumanji-Robin-Williams/dp/B00BZBQ4GW/ref=sr_1_1?dchild=1&keywords=Jumanji+1995&qid=1619255969&s=movies-tv&sr=1-1'

In [62]:
!pip3 install urlparse

ERROR: Could not find a version that satisfies the requirement urlparse (from versions: none)
ERROR: No matching distribution found for urlparse


In [92]:
from w3lib.url import safe_url_string
url = safe_url_string('http://example.org/Ñöñ-ÅŞÇİİ/', encoding="utf-8")
print(url)

http://example.org/%C3%91%C3%B6%C3%B1-%C3%85%C5%9E%C3%87%C4%B0%C4%B0/


In [106]:
!pip install requests-random-user-agent

     |████████████████████████████████| 47 kB 1.6 MB/s eta 0:00:01


In [15]:
!pip3 install lxml

     |████████████████████████████████| 4.6 MB 609 kB/s eta 0:00:01


In [5]:
import threading
import requests
from lxml import etree
from urllib import request
import os

import re

class PageUrl(threading.Thread):
    def __init__(self, l, *args, **kwargs):
        super(PageUrl, self).__init__(*args,**kwargs)
        self.l = l

    def run(self):
        runGetPage(self.l)
        
class Price(threading.Thread):
    def __init__(self, l, *args, **kwargs):
        super(Price, self).__init__(*args,**kwargs)
        self.l = l

    def run(self):
        runGetPrice(self.l)

In [22]:
cursor.execute("SELECT * FROM prices;")
movie_list = cursor.fetchall()
PageUrl(movie_list).start()

Processing row 1:   0%|          | 0/3883 [00:00<?, ?it/s]

Exception at row 0
'NoneType' object is not subscriptable


Processing row 248:   4%|▍         | 170/3883 [00:00<00:02, 1694.22it/s]

Exception at row 1
'NoneType' object is not subscriptable
Exception at row 2
'NoneType' object is not subscriptable
Exception at row 3
'NoneType' object is not subscriptable
Exception at row 4
'NoneType' object is not subscriptable
Exception at row 5
'NoneType' object is not subscriptable
Exception at row 6
'NoneType' object is not subscriptable
Exception at row 7
'NoneType' object is not subscriptable
Exception at row 8
'NoneType' object is not subscriptable
Exception at row 9
'NoneType' object is not subscriptable
Exception at row 10
'NoneType' object is not subscriptable
Exception at row 11
'NoneType' object is not subscriptable
Exception at row 12
'NoneType' object is not subscriptable
Exception at row 13
'NoneType' object is not subscriptable
Exception at row 14
'NoneType' object is not subscriptable
Exception at row 15
'NoneType' object is not subscriptable
Exception at row 16
'NoneType' object is not subscriptable
Exception at row 17
'NoneType' object is not subscriptable
Except

Processing row 420:   9%|▉         | 340/3883 [00:00<00:03, 1121.16it/s]

...

Exception at row 2619
'NoneType' object is not subscriptable
Exception at row 2620
'NoneType' object is not subscriptable
Exception at row 2621
'NoneType' object is not subscriptable
Exception at row 2622
'NoneType' object is not subscriptable
Exception at row 2623
'NoneType' object is not subscriptable
Exception at row 2624
'NoneType' object is not subscriptable
Exception at row 2625
'NoneType' object is not subscriptable
Exception at row 2626
'NoneType' object is not subscriptable
Exception at row 2627
'NoneType' object is not subscriptable
Exception at row 2628
'NoneType' object is not subscriptable
Exception at row 2629
'NoneType' object is not subscriptable
Exception at row 2630
'NoneType' object is not subscriptable
Exception at row 2631
'NoneType' object is not subscriptable
Exception at row 2632
'NoneType' object is not subscriptable
Exception at row 2633
'NoneType' object is not subscriptable
Exception at row 2634
'NoneType' object is not subscriptable
Exception at row 2635
'N

Processing row 3003:  77%|███████▋  | 2987/3883 [00:03<00:00, 1017.53it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Processing row 3857:  98%|█████████▊| 3793/3883 [00:03<00:00, 1270.30it/s]

'NoneType' object is not subscriptable
Exception at row 3577
'NoneType' object is not subscriptable
Exception at row 3578
'NoneType' object is not subscriptable
Exception at row 3579
'NoneType' object is not subscriptable
Exception at row 3580
'NoneType' object is not subscriptable
Exception at row 3581
'NoneType' object is not subscriptable
Exception at row 3582
'NoneType' object is not subscriptable
Exception at row 3583
'NoneType' object is not subscriptable
Exception at row 3584
'NoneType' object is not subscriptable
Exception at row 3585
'NoneType' object is not subscriptable
Exception at row 3586
'NoneType' object is not subscriptable
Exception at row 3587
'NoneType' object is not subscriptable
Exception at row 3588
'NoneType' object is not subscriptable
Exception at row 3589
'NoneType' object is not subscriptable
Exception at row 3590
'NoneType' object is not subscriptable
Exception at row 3591
'NoneType' object is not subscriptable
Exception at row 3592
'NoneType' object is not

Processing row 3882: 100%|██████████| 3883/3883 [00:03<00:00, 1022.59it/s]


Exception at row 3857
'NoneType' object is not subscriptable
Exception at row 3858
'NoneType' object is not subscriptable
Exception at row 3859
'NoneType' object is not subscriptable
Exception at row 3860
'NoneType' object is not subscriptable
Exception at row 3861
'NoneType' object is not subscriptable
Exception at row 3862
'NoneType' object is not subscriptable
Exception at row 3863
'NoneType' object is not subscriptable
Exception at row 3864
'NoneType' object is not subscriptable
Exception at row 3865
'NoneType' object is not subscriptable
Exception at row 3866
'NoneType' object is not subscriptable
Exception at row 3867
'NoneType' object is not subscriptable
Exception at row 3868
'NoneType' object is not subscriptable
Exception at row 3869
'NoneType' object is not subscriptable
Exception at row 3870
'NoneType' object is not subscriptable
Exception at row 3871
'NoneType' object is not subscriptable
Exception at row 3872
'NoneType' object is not subscriptable
Exception at row 3873
'N

In [6]:
cursor.execute("SELECT * FROM prices;")
movie_list = cursor.fetchall()
WORKERS_NUM = 5
split_list = np.array_split(movie_list, 5)

In [7]:
for i in range(WORKERS_NUM):
    t = PageUrl(split_list[i])
    t.start()

Processing row 16:   2%|▏         | 16/777 [00:17<18:23,  1.45s/it]

In [21]:
cursor.execute('SELECT * FROM prices WHERE pageUrl IS NOT NULL')
len(cursor.fetchall())

612

In [6]:
cursor.execute('SELECT * FROM prices')
len(cursor.fetchall())

27278

In [58]:
cursor.execute('SELECT * FROM prices WHERE pageUrl IS NOT NULL')
len(cursor.fetchall())

Processing row 2912:  53%|█████▎    | 2912/5455 [1:07:56<1:03:31,  1.50s/it]

Exception at row 2911
'NoneType' object has no attribute 'find'


26475

Processing row 3681:  67%|██████▋   | 3681/5456 [1:07:57<56:59,  1.93s/it]

Exception at row 3680
'NoneType' object has no attribute 'find'


...

Processing row 3241:  59%|█████▉    | 3241/5455 [1:08:13<57:29,  1.56s/it]  

Exception at row 3240
'NoneType' object has no attribute 'find'


Processing row 3244:  59%|█████▉    | 3242/5455 [1:08:13<43:06,  1.17s/it]

Exception at row 3241
'NoneType' object has no attribute 'find'


Processing row 3245:  59%|█████▉    | 3245/5455 [1:08:13<31:05,  1.18it/s]

In [23]:
getMoviePrice("https://www.amazon.com/Superbad-Unrated-Jonah-Hill/dp/B00EFCHT08/ref=sr_1_1?dchild=1&keywords=Superbad+2007&qid=1619260061&s=movies-tv&sr=1-1")

6.99

In [46]:
lst = [str(x) for x in obj.body.find_all('button') if 'Buy' in ]

In [58]:
re.findall(r'(?<=\$)\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})', "$6.99")

['6.99']

In [9]:
cursor.execute("SELECT * FROM prices WHERE price IS NULL;")
movie_list = cursor.fetchall()
WORKERS_NUM = 5
split_list = np.array_split(movie_list, 5)

In [10]:
for i in range(WORKERS_NUM):
    t = Price(split_list[i])
    t.start()

Processing row 3:   0%|          | 3/3168 [00:01<30:30,  1.73it/s]t]

Exception at row 0
'NoneType' object has no attribute 'find'
Exception at row 2
'NoneType' object has no attribute 'find'


Processing row 1:   0%|          | 1/3169 [00:02<2:17:07,  2.60s/it]

Exception at row 0
'NoneType' object has no attribute 'find'


Processing row 4:   0%|          | 4/3168 [00:03<47:43,  1.11it/s]

...

Exception at row 7
'NoneType' object has no attribute 'find'


Processing row 5:   0%|          | 5/3169 [00:20<4:08:44,  4.72s/it]

Exception at row 7
'NoneType' object has no attribute 'find'
Exception at row 4
'NoneType' object has no attribute 'find'


Processing row 5:   0%|          | 5/3168 [00:21<3:45:39,  4.28s/it]

Exception at row 4
'NoneType' object has no attribute 'find'


'<button class="_1ovr-S _15Ikr8 _20uSE4 _2X_Irl tvod-button av-button" type="submit"><span class="_36qUej">Buy <span class="dv-conditional-linebreak"><br/></span><strong>HD</strong> $6.99</span></button>'

In [21]:
cursor.execute('SELECT * FROM prices WHERE price IS NOT NULL')
len(cursor.fetchall())

11524

Processing row 238:   8%|▊         | 238/3168 [07:04<1:33:55,  1.92s/it]

Exception at row 237
'NoneType' object has no attribute 'find'


Processing row 234:   7%|▋         | 234/3169 [07:04<1:25:38,  1.75s/it]

Exception at row 233
'NoneType' object has no attribute 'find'


...

Exception at row 294
<urlopen error timed out>


Processing row 258:   8%|▊         | 258/3169 [08:10<3:56:28,  4.87s/it]

Exception at row 294
<urlopen error timed out>
Exception at row 257
<urlopen error timed out>


Processing row 228:   7%|▋         | 228/3168 [08:11<3:54:57,  4.80s/it]

Exception at row 259
<urlopen error timed out>
Exception at row 227
<urlopen error timed out>


In [5]:
url = "https://www.amazon.com/Johnny-Cash-Live-Montreux-1994/dp/B003OBFVJW/ref=sr_1_1?dchild=1&keywords=Blue+Sky+1994&qid=1619257244&s=movies-tv&sr=1-1"
getMoviePrice(url)

9.99

In [65]:
removeBrackets("Citizen Gangster (2011)")

'Citizen Gangster 2011'

Processing row 4775:  88%|████████▊ | 4775/5455 [1:47:44<26:06,  2.30s/it]

Exception at row 4774
'NoneType' object has no attribute 'find'


Processing row 4744:  87%|████████▋ | 4743/5456 [1:47:44<32:30,  2.74s/it]

Exception at row 4742
'NoneType' object has no attribute 'find'


...

Exception at row 5452
'NoneType' object has no attribute 'find'


Processing row 5454: 100%|█████████▉| 5454/5455 [2:03:51<00:01,  1.57s/it]

Exception at row 5453
'NoneType' object has no attribute 'find'


Processing row 5454: 100%|██████████| 5455/5455 [2:03:52<00:00,  1.36s/it]

Exception at row 5454
'NoneType' object has no attribute 'find'
